In [ ]:
import os
import torch
import pandas as pd
import wandb
from ultralytics import YOLO


CLASS_NAMES = {0: "fake", 1: "real"}

def log_training_results(csv_path, run_name="Anti_spoof"):
    # Log metrics from the CSV file YOLO produces
    df = pd.read_csv(csv_path)

    for _, row in df.iterrows():
        wandb.log({
            "epoch": int(row["epoch"]) + 1,
            "train_loss": row["train/box_loss"] + row["train/cls_loss"] + row["train/dfl_loss"],
            "val_loss": row["val/box_loss"] + row["val/cls_loss"] + row["val/dfl_loss"],
            "val_precision": row.get("metrics/precision(B)", None),
            "val_recall": row.get("metrics/recall(B)", None),
            "val_mAP50": row.get("metrics/mAP50(B)", None),
            "val_mAP50-95": row.get("metrics/mAP50-95(B)", None)
        })

def train_and_evaluate():
    # Check device
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print(" Using GPU:", torch.cuda.get_device_name(0))
        print(" Total GPU memory (GB):", round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2))
    else:
        print(" Using CPU")

    # Init W&B run
    wandb.login(key="f8b90505309db381cb58d016f8670403992ffad9")
    run = wandb.init(project="anti-spoofing", name="Anti_spoof", resume="allow")

    # Train YOLO
    model = YOLO("yolov8n.pt")
    model.train(
        data="E:/STUDIES/Prj/Anti_Spoof/DATASET/Split_Data1/data.yaml",
        epochs=100,
        imgsz=416,
        batch=16,
        patience=10,
        project="Anti_Spoofing_YoLo",
        name="Anti_spoof",
        val=True,
        device=0,
        verbose=True,
        plots=True,
        workers=0,
        close_mosaic=0
    )

    # Log training metrics from results.csv
    results_csv = "E:/STUDIES/Prj/Anti_Spoof/Anti_Spoofing_YoLo/Anti_spoof6/results.csv"
    if os.path.exists(results_csv):
        log_training_results(results_csv)
    else:
        print(f"CSV file not found at {results_csv}")

    print("Running model.val()...")
    metrics = model.val()

    # Log evaluation metrics to W&B
    wandb.log({
        "val/box_loss": metrics.box.loss,
        "val/cls_loss": metrics.box.cls,
        "val/mAP50": metrics.box.map50,
        "val/mAP50-95": metrics.box.map
    })

    # Finish the W&B run
    wandb.finish()

if __name__ == "__main__":
    train_and_evaluate()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\annho\_netrc


CUDA available: True
 Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU
 Total GPU memory (GB): 4.0


wandb: Currently logged in as: annho4110 (annho4110-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


New https://pypi.org/project/ultralytics/8.3.167 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.160  Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:/STUDIES/Prj/Anti_Spoof/DATASET/Split_Data1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Anti_

train: Scanning E:\STUDIES\Prj\Anti_Spoof\DATASET\Split_Data1\train\labels... 5637 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5637/5637 [00:02<00:00, 2319.50it/s]


...

                   all       1611       1616          1      0.999      0.995      0.928
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 89, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



99 epochs completed in 2.470 hours.
Optimizer stripped from Anti_Spoofing_YoLo\Anti_spoof6\weights\last.pt, 6.2MB
Optimizer stripped from Anti_Spoofing_YoLo\Anti_spoof6\weights\best.pt, 6.2MB

Validating Anti_Spoofing_YoLo\Anti_spoof6\weights\best.pt...
Ultralytics 8.3.160  Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:16<00:00,  3.11it/s]


                   all       1611       1616          1      0.999      0.995      0.928
                  fake        806        806          1      0.999      0.995      0.922
                  real        805        810          1      0.999      0.995      0.934
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to Anti_Spoofing_YoLo\Anti_spoof6

Loading best model for evaluation...


FileNotFoundError: [Errno 2] No such file or directory: 'runs\\detect\\Anti_spoof\\weights\\best.pt'